### Exercises
    Select a dataset with a continuous target variable.

    Be sure your data is prepared (no missing values, numeric datatypes) and split into samples.

    Work through all of the steps outlined in the lesson, from setting the baseline to selected a model and evaluating the final model on your test data.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydataset
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
from sklearn.metrics import mean_squared_error, r2_score
import prepare_telco
import wrangle
import os
from env import user, password, host
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression 
from math import sqrt
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import prepare_telco
from scipy.stats import pearsonr, spearmanr
from statsmodels.formula.api import ols

In [16]:
#load wrangled dataset
df=wrangle.wrangle_zillow()

In [18]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094583 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2094583 non-null  int32  
 1   bathroomcnt                   2094583 non-null  float64
 2   calculatedfinishedsquarefeet  2094583 non-null  int32  
 3   taxvaluedollarcnt             2094583 non-null  int32  
 4   yearbuilt                     2094583 non-null  int32  
 5   taxamount                     2094583 non-null  float64
 6   fips                          2094583 non-null  int32  
dtypes: float64(2), int32(5)
memory usage: 87.9 MB


In [19]:
#new column total bill divided by size
df['cost_per_sqft'] = (df['taxvaluedollarcnt']/df['calculatedfinishedsquarefeet']).round(2)

In [20]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094583 entries, 4 to 2152862
Data columns (total 8 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2094583 non-null  int32  
 1   bathroomcnt                   2094583 non-null  float64
 2   calculatedfinishedsquarefeet  2094583 non-null  int32  
 3   taxvaluedollarcnt             2094583 non-null  int32  
 4   yearbuilt                     2094583 non-null  int32  
 5   taxamount                     2094583 non-null  float64
 6   fips                          2094583 non-null  int32  
 7   cost_per_sqft                 2094583 non-null  float64
dtypes: float64(3), int32(5)
memory usage: 103.9 MB


In [22]:
#avg sqft per number of rooms (bedroom+bathroom)
df['sqft_room_ratio'] = (df['calculatedfinishedsquarefeet']/(df['bedroomcnt']+df['bathroomcnt'])).round(2)

In [23]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094583 entries, 4 to 2152862
Data columns (total 9 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2094583 non-null  int32  
 1   bathroomcnt                   2094583 non-null  float64
 2   calculatedfinishedsquarefeet  2094583 non-null  int32  
 3   taxvaluedollarcnt             2094583 non-null  int32  
 4   yearbuilt                     2094583 non-null  int32  
 5   taxamount                     2094583 non-null  float64
 6   fips                          2094583 non-null  int32  
 7   cost_per_sqft                 2094583 non-null  float64
 8   sqft_room_ratio               2094583 non-null  float64
dtypes: float64(4), int32(5)
memory usage: 119.9 MB


In [10]:
#avg sqft per number of rooms (bedroom+bathroom)
df['sqft_room_ratio'] = (df['calculatedfinishedsquarefeet']/(df['bedroomcnt']+df['bathroomcnt'])).round(2)

In [68]:
(df.bedroomcnt == 0).value_counts()


False    2090302
True        4281
Name: bedroomcnt, dtype: int64

In [30]:
(df.bathroomcnt == 0).value_counts()

False    2090400
True        4183
Name: bathroomcnt, dtype: int64

In [64]:
df['bedroomcnt']==0

4          False
6          False
7          False
11          True
14          True
           ...  
2152856    False
2152858    False
2152859    False
2152861    False
2152862    False
Name: bedroomcnt, Length: 2094583, dtype: bool

In [65]:
df['bathroomcnt']==0

4          False
6          False
7          False
11          True
14          True
           ...  
2152856    False
2152858    False
2152859    False
2152861    False
2152862    False
Name: bathroomcnt, Length: 2094583, dtype: bool

In [66]:
(df['bedroomcnt']==0)&(df['bathroomcnt']==0)

4          False
6          False
7          False
11          True
14          True
           ...  
2152856    False
2152858    False
2152859    False
2152861    False
2152862    False
Length: 2094583, dtype: bool

In [71]:
df[(df['bedroomcnt']==0)&(df['bathroomcnt']==0)].describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,cost_per_sqft,sqft_room_ratio
count,3274.0,3274.0,3274.000000,3.274000e+03,3274.000000,3274.000000,3274.000000,3274.000000,3274.0
mean,0.0,0.0,997.124007,1.670785e+05,1952.376298,2159.928177,6040.145388,226.650776,inf
std,0.0,0.0,1616.759301,2.664791e+05,25.048955,3664.138334,11.850953,647.932280,NaN
min,0.0,0.0,1.000000,2.200000e+01,1890.000000,6.340000,6037.000000,0.030000,inf
25%,0.0,0.0,280.000000,1.589850e+04,1931.000000,330.997500,6037.000000,40.992500,NaN
50%,0.0,0.0,510.000000,5.771200e+04,1952.000000,895.330000,6037.000000,98.500000,NaN
75%,0.0,0.0,1079.000000,2.020382e+05,1970.000000,2583.125000,6037.000000,232.277500,NaN
max,0.0,0.0,28609.000000,1.964300e+06,2016.000000,93201.760000,6111.000000,28091.000000,inf


In [72]:
df[(df['bedroomcnt']!=0)&(df['bathroomcnt']!=0)].describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,cost_per_sqft,sqft_room_ratio
count,2.089393e+06,2.089393e+06,2.089393e+06,2.089393e+06,2.089393e+06,2.089393e+06,2.089393e+06,2.089393e+06,2.089393e+06
mean,3.277123e+00,2.186746e+00,1.798880e+03,3.990427e+05,1.960681e+03,4.914563e+03,6.048368e+03,2.243419e+02,3.257907e+02
std,8.788937e-01,8.712688e-01,1.026000e+03,3.134196e+05,2.184710e+01,3.638174e+03,2.041250e+01,7.994765e+02,1.169436e+02
min,1.000000e+00,5.000000e-01,1.000000e+00,2.200000e+01,1.801000e+03,1.648000e+01,6.037000e+03,1.000000e-02,1.000000e-01
25%,3.000000e+00,2.000000e+00,1.252000e+03,1.866240e+05,1.949000e+03,2.512480e+03,6.037000e+03,1.237900e+02,2.672900e+02
50%,3.000000e+00,2.000000e+00,1.608000e+03,3.225560e+05,1.958000e+03,4.044510e+03,6.037000e+03,1.999000e+02,3.113300e+02
75%,4.000000e+00,3.000000e+00,2.160000e+03,5.170600e+05,1.975000e+03,6.214460e+03,6.059000e+03,2.941200e+02,3.686700e+02
max,6.000000e+00,6.000000e+00,9.525760e+05,1.999995e+06,2.016000e+03,1.003793e+05,6.111000e+03,6.097510e+05,1.190720e+05


at this point after digging into whether or not i can or should impute the values of the averages based on squarefootage or just drop them

In [73]:
df.drop(df.loc[df['bedroomcnt']==0].index, inplace=True)

In [75]:
df.drop(df.loc[df['bathroomcnt']==0].index, inplace=True)

In [76]:
(df.bedroomcnt == 0).value_counts()

False    2089393
Name: bedroomcnt, dtype: int64

In [77]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,cost_per_sqft,sqft_room_ratio
4,4,2.0,3633,296425,2005,6941.39,6037,81.59,605.50
6,3,4.0,1620,847770,2011,10244.94,6037,523.31,231.43
7,3,2.0,2077,646760,1926,7924.68,6037,311.39,415.40
18,3,1.0,1244,169471,1950,2532.88,6037,136.23,311.00
19,3,2.0,1300,233266,1950,3110.99,6037,179.44,260.00


In [ ]:
#split the data

In [ ]:
#scale

In [ ]:
#establish baseline

In [ ]:
#separate target

In [ ]:
#fit model to train

In [ ]:
#build df with model predictions

In [ ]:
# # We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
# y_train = pd.DataFrame(y_train)
# y_validate = pd.DataFrame(y_validate)

# # 1. Predict G3_pred_mean
# G3_pred_mean = y_train['G3'].mean()
# y_train['G3_pred_mean'] = G3_pred_mean
# y_validate['G3_pred_mean'] = G3_pred_mean

# # 2. compute G3_pred_median
# G3_pred_median = y_train['G3'].median()
# y_train['G3_pred_median'] = G3_pred_median
# y_validate['G3_pred_median'] = G3_pred_median

# # 3. RMSE of G3_pred_mean
# rmse_train = mean_squared_error(y_train.G3, y_train.G3_pred_mean)**(1/2)
# rmse_validate = mean_squared_error(y_validate.G3, y_validate.G3_pred_mean)**(1/2)

# print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
#       "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

# # 4. RMSE of G3_pred_median
# rmse_train = mean_squared_error(y_train.G3, y_train.G3_pred_median)**(1/2)
# rmse_validate = mean_squared_error(y_validate.G3, y_validate.G3_pred_median)**(1/2)

# print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train, 2), 
#       "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

In [ ]:
# # plot to visualize actual vs predicted. 
# plt.hist(y_train.G3, color='blue', alpha=.5, label="Actual Final Grades")
# plt.hist(y_train.G3_pred_mean, bins=1, color='red', alpha=.5, rwidth=100, label="Predicted Final Grades - Mean")
# plt.hist(y_train.G3_pred_median, bins=1, color='orange', alpha=.5, rwidth=100, label="Predicted Final Grades - Median")
# plt.xlabel("Final Grade (G3)")
# plt.ylabel("Number of Students")
# plt.legend()
# plt.show()

In [ ]:
# Identify Baseline: establish baseline prediction and accuracy
# Modeling
# Train
# Validate
# Test
# Predict on Test Model

In [ ]:
#data needs to be cleaned 
# Convert binary categorical variables to numeric
    tips['sex'] = tips.sex.map({'Female': 1, 'Male': 0})
    tips['smoker'] = tips.smoker.map({'Yes': 1, 'No': 0})
    
    
    # Get dummies for non-binary categorical variables
    dummy_df = pd.get_dummies(tips[['day', \
                              'time']], dummy_na=False, \
                              drop_first=True)
    
    # Concatenate dummy dataframe to original 
    tips = pd.concat([tips, dummy_df], axis=1)

In [79]:
sql = '''
    SELECT
        bedroomcnt AS bedrooms,
        bathroomcnt AS bathrooms,
        calculatedfinishedsquarefeet AS sq_ft,
        taxvaluedollarcnt AS tax_value,
        yearbuilt AS year_built,
        taxamount AS tax_amnt,
        lotsizesquarefeet AS lot_size,
        fips
    FROM
        properties_2017
       JOIN propertylandusetype using (propertylandusetypeid)
       JOIN predictions_2017 USING(parcelid)
    WHERE propertylandusedesc in ("Single Family Residential",
                                  "Inferred Single Family Residential")
       AND transactiondate LIKE "2017%%";
    '''
return pd.read_sql(sql, get_db_url('zillow'))


SyntaxError: 'return' outside function (809998586.py, line 19)